# Simple IMPT plan optimization

In [1]:
import math
import os
import sys

import numpy as np
from matplotlib import pyplot as plt

from opentps.core.data.images import CTImage
from opentps.core.data.images import ROIMask
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.data import Patient
from opentps.core.data.plan import FidObjective
from opentps.core.io import mcsquareIO
from opentps.core.io.scannerReader import readScanner
from opentps.core.io.serializedObjectIO import saveRTPlan, loadRTPlan
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D, resampleImage3D
from opentps.core.processing.planOptimization.planOptimization import IMPTPlanOptimizer

15/07/2024 01:27:26 PM - root - INFO - Loading logging configuration: C:\Users\cgaban\opentps\opentps_core\opentps\core\config\logger\logging_config.json
15/07/2024 01:27:26 PM - opentps.core._loggingConfig - INFO - Log level set: INFO
15/07/2024 01:27:28 PM - opentps.core.processing.imageProcessing.cupyImageProcessing - WARNING - Cannot import Cupy module
15/07/2024 01:27:28 PM - opentps.core.processing.registration.registrationMorphons - WARNING - cupy not found.
15/07/2024 01:27:28 PM - opentps.core.processing.registration.morphonsCupy - WARNING - cupy not found.
15/07/2024 01:27:28 PM - opentps.core.processing.C_libraries.libInterp3_wrapper - WARNING - cupy not found.
15/07/2024 01:27:29 PM - opentps.core.processing.planOptimization.solvers.lp - WARNING - Ignore the following warning if not using Gurobi linear optimizer. Gurobi not required for most features provided in OpenTPS
15/07/2024 01:27:29 PM - opentps.core.processing.planOptimization.solvers.lp - WARNING - No module Gurobi

## CT calibration and BDL

In [2]:
ctCalibration = readScanner(DoseCalculationConfig().scannerFolder)
bdl = mcsquareIO.readBDL(DoseCalculationConfig().bdlFile)

## Create synthetic CT and ROI

In [3]:
patient = Patient()
patient.name = 'Patient'

ctSize = 150

ct = CTImage()
ct.name = 'CT'
ct.patient = patient


huAir = -1024.
huWater = ctCalibration.convertRSP2HU(1.)
data = huAir * np.ones((ctSize, ctSize, ctSize))
data[:, 50:, :] = huWater
ct.imageArray = data

roi = ROIMask()
roi.patient = patient
roi.name = 'TV'
roi.color = (255, 0, 0) # red
data = np.zeros((ctSize, ctSize, ctSize)).astype(bool)
data[100:120, 100:120, 100:120] = True
roi.imageArray = data

## Configure dose engine

In [4]:
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.nbPrimaries = 5e4
mc2.ctCalibration = ctCalibration

mc2._independentScoringGrid = True
scoringSpacing = [2, 2, 2]
mc2._scoringVoxelSpacing = scoringSpacing

## Design plan

In [6]:
beamNames = ["Beam1"]
gantryAngles = [0.]
couchAngles = [0.]

planInit = PlanDesign()
planInit.ct = ct
planInit.targetMask = roi
planInit.gantryAngles = gantryAngles
planInit.beamNames = beamNames
planInit.couchAngles = couchAngles
planInit.calibration = ctCalibration
planInit.spotSpacing = 6.0
planInit.layerSpacing = 6.0
planInit.targetMargin = 0.0
planInit.setScoringParameters(scoringSpacing=[2, 2, 2], adapt_gridSize_to_new_spacing=True)

plan = planInit.buildPlan()  # Spot placement
plan.PlanName = "NewPlan"

beamlets = mc2.computeBeamlets(ct, plan, roi=[roi])
plan.planDesign.beamlets = beamlets
# doseImageRef = beamlets.toDoseImage()

15/07/2024 01:34:09 PM - opentps.core.data.plan._planDesign - INFO - Building plan ...
15/07/2024 01:34:09 PM - opentps.core.processing.planOptimization.planInitializer - INFO - Target is dilated using a margin of 0.0 mm. This process might take some time.
15/07/2024 01:34:09 PM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.
15/07/2024 01:34:14 PM - opentps.core.data.plan._planDesign - INFO - New plan created in 5.5135955810546875 sec
15/07/2024 01:34:14 PM - opentps.core.data.plan._planDesign - INFO - Number of spots: 84
15/07/2024 01:34:14 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Beamlet calculation
15/07/2024 01:34:21 PM - opentps.core.io.mhdIO - INFO - Write MHD file: C:\Users\cgaban\openTPS_workspace\Simulations\MCsquare_simulation\CT.mhd
15/07/2024 01:34:21 PM - opentps.core.io.mcsquareIO - INFO - Write plan: C:\Users\cgaban\openTPS_workspace\Simulations\MCsquare_simulation\Plan

## Objectives

In [7]:
plan.planDesign.objectives = ObjectivesList()
plan.planDesign.objectives.setTarget(roi.name, 20.0)

plan.planDesign.objectives.fidObjList = []
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.DMAX, 20.0, 1.0)
plan.planDesign.objectives.addFidObjective(roi, FidObjective.Metrics.DMIN, 20.5, 1.0)

## Optimize plan

In [13]:
solver = IMPTPlanOptimizer(method='Scipy-LBFGS', plan=plan, maxit=50)
doseImage, ps = solver.optimize()

15/07/2024 02:56:33 PM - opentps.core.processing.planOptimization.planOptimization - INFO - Prepare optimization ...
15/07/2024 02:56:33 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - Iteration 1 of Scipy-L-BFGS-B
15/07/2024 02:56:33 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - objective = 4.509621e+01  
15/07/2024 02:56:33 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - Iteration 2 of Scipy-L-BFGS-B
15/07/2024 02:56:33 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - objective = 3.423027e+01  
15/07/2024 02:56:33 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - Iteration 3 of Scipy-L-BFGS-B
15/07/2024 02:56:33 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - objective = 2.884240e+01  
15/07/2024 02:56:34 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - Iteration 4 of Scipy-L-BFGS-B
15/07/2024 02:56:34 PM - opentps.core.processing.planOptimization.

## Final dose computation

In [15]:
mc2.nbPrimaries = 1e7
doseImage = mc2.computeDose(ct, plan)

02/08/2023 02:20:26 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Dose calculation
02/08/2023 02:20:26 PM - opentps.core.io.mhdIO - INFO - Write MHD file: C:\Users\romai\openTPS_workspace\Simulations\MCsquare_simulation\CT.mhd
02/08/2023 02:20:26 PM - opentps.core.io.mcsquareIO - INFO - Write plan: C:\Users\romai\openTPS_workspace\Simulations\MCsquare_simulation\PlanPencil.txt
02/08/2023 02:20:27 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


## Plots

In [12]:
 # Compute DVH on resampled contour
roiResampled = resampleImage3D(roi, origin=ct.origin, spacing=scoringSpacing)
target_DVH = DVH(roiResampled, doseImage)
print('D95 = ' + str(target_DVH.D95) + ' Gy')
print('D5 = ' + str(target_DVH.D5) + ' Gy')
print('D5 - D95 =  {} Gy'.format(target_DVH.D5 - target_DVH.D95))

# center of mass
roi = resampleImage3DOnImage3D(roi, ct)
COM_coord = roi.centerOfMass
COM_index = roi.getVoxelIndexFromPosition(COM_coord)
Z_coord = COM_index[2]

img_ct = ct.imageArray[:, :, Z_coord].transpose(1, 0)
contourTargetMask = roi.getBinaryContourMask()
img_mask = contourTargetMask.imageArray[:, :, Z_coord].transpose(1, 0)
img_dose = resampleImage3DOnImage3D(doseImage, ct)
img_dose = img_dose.imageArray[:, :, Z_coord].transpose(1, 0)

#Output path
output_path = 'Output'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Display dose
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(img_ct, cmap='gray')
ax[0].imshow(img_mask, alpha=.2, cmap='binary')  # PTV
dose = ax[0].imshow(img_dose, cmap='jet', alpha=.2)
plt.colorbar(dose, ax=ax[0])
ax[1].plot(target_DVH.histogram[0], target_DVH.histogram[1], label=target_DVH.name)
ax[1].set_xlabel("Dose (Gy)")
ax[1].set_ylabel("Volume (%)")
plt.grid(True)
plt.legend()
plt.show()

plt.savefig(os.path.join(output_path, 'SimpleOpti1.png'),format = 'png')
plt.close()

D95 = 15.05126953125 Gy
D5 = 23.62060546875 Gy
D5 - D95 =  8.5693359375 Gy
15/07/2024 02:21:38 PM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.


![png](\assets\img_notebooks\SimpleOpti1.png)